# Ingest SEC DERA data into Trino pipeline

Copyright (C) 2021 OS-Climate

This sample shows:
* How to create schemas and tables via the Trino / SQLAlchemy on an underlying Iceberg data volume
* Apache Iceberg ACID transaction and time travel capabilities used for data set versioning


Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

Contributed by Michael Tiemann (Github: MichaelTiemannOSC)

In [2]:
%%capture pipoutput
%pip install boto3 python-dotenv
%pip install --upgrade sqlalchemy==1.3 sqlalchemy-trino
%pip install pandas pyarrow fastparquet
%pip install anytree
%pip install osc-ingest-tools

In [3]:
from dotenv import dotenv_values, load_dotenv
from osc_ingest_trino import *
import os
import pathlib
import pandas as pd
import uuid
import trino
from sqlalchemy.engine import create_engine

Load Environment Variables

In [18]:
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

In [19]:
import boto3

s3_resource = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)

Create a simple data frame for testing

In [23]:
import io

buffer = io.BytesIO()
sub_file = s3_resource.Object(os.environ['S3_LANDING_BUCKET'],'SEC-DERA/2020q4/sub.txt')
sub_file.download_fileobj(buffer)
buffer.seek(0)
df = pd.read_csv(buffer, header=0, sep='\t', nrows=100, engine='c')
df

,adsh,cik,name,sic,countryba,stprba,cityba,zipba,bas1,bas2,...,period,fy,fp,filed,accepted,prevrpt,detail,instance,nciks,aciks
0,0000010254-20-000105,10254,EARTHSTONE ENERGY INC,1311,US,TX,THE WOODLANDS,77380,1400 WOODLOCH FOREST DRIVE,SUITE 300,...,20200930,NaN,NaN,20201001,2020-10-01 16:05:00.0,0,0,form8-kxbbredeterminat_htm.xml,1,NaN
1,0000016918-20-000236,16918,"CONSTELLATION BRANDS, INC.",2080,US,NY,VICTOR,14564,207 HIGH POINT DRIVE,BUILDING 100,...,20200930,NaN,NaN,20201001,2020-10-01 08:04:00.0,0,0,stz-20200930_htm.xml,1,NaN
2,0000016918-20-000237,16918,"CONSTELLATION BRANDS, INC.",2080,US,NY,VICTOR,14564,207 HIGH POINT DRIVE,BUILDING 100,...,20200831,2021.0,Q2,20201001,2020-10-01 14:33:00.0,0,1,stz-20200831_htm.xml,1,NaN
3,0000022444-20-000035,22444,COMMERCIAL METALS CO,3312,US,TX,IRVING,75039,"6565 N. MACARTHUR BLVD., SUITE 800",P O BOX 1046,...,20200930,NaN,NaN,20201001,2020-10-01 09:45:00.0,0,0,cmc-20201001_htm.xml,1,NaN
4,0000034088-20-000081,34088,EXXON MOBIL CORP,2911,US,TX,IRVING,75039-2298,5959 LAS COLINAS BLVD,NaN,...,20200930,NaN,NaN,20201001,2020-10-01 07:02:00.0,0,0,r8k100120_htm.xml,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0001193125-20-260523,811532,CEDAR FAIR L P,7990,US,OH,SANDUSKY,44870,ONE CEDAR POINT DRIVE,NaN,...,20200930,NaN,NaN,20201001,2020-10-01 07:52:00.0,0,0,d69635d8k_htm.xml,1,NaN
96,0001193125-20-260538,700564,FULTON FINANCIAL CORP,6021,US,PA,LANCASTER,17604,ONE PENN SQ,PO BOX 4887,...,20200930,NaN,NaN,20201001,2020-10-01 08:01:00.0,0,0,d22517d8k_htm.xml,1,NaN
97,0001193125-20-260540,816956,CONMED CORP,3845,US,NY,UTICA,13502,525 FRENCH ROAD,NaN,...,20200930,NaN,NaN,20201001,2020-10-01 08:01:00.0,0,0,d934719d8k_htm.xml,1,NaN
98,0001193125-20-260541,1158172,"COMSCORE, INC.",7389,US,VA,RESTON,20190,11950 DEMOCRACY DRIVE,SUITE 600,...,20200930,NaN,NaN,20201001,2020-10-01 08:01:00.0,0,0,d153918d8k_htm.xml,1,NaN


Create custom meta data and declare variable for schema and table for the data set

In [24]:
df.columns

Index(['adsh', 'cik', 'name', 'sic', 'countryba', 'stprba', 'cityba', 'zipba',
       'bas1', 'bas2', 'baph', 'countryma', 'stprma', 'cityma', 'zipma',
       'mas1', 'mas2', 'countryinc', 'stprinc', 'ein', 'former', 'changed',
       'afs', 'wksi', 'fye', 'form', 'period', 'fy', 'fp', 'filed', 'accepted',
       'prevrpt', 'detail', 'instance', 'nciks', 'aciks'],
      dtype='object')

In [27]:
custom_meta_content = {
    'dataset_key': 'SEC-DERA',
    'title': 'SEC DERA Disclosures',
    'description': 
    '''The DERA Financial Statement Data Sets provide numeric information from the face financials of all financial statements.
    
    This data is extracted from exhibits to corporate financial reports filed with the Commission using eXtensible Business Reporting Language (XBRL).  As compared to the more extensive Financial Statement and Notes Data Sets, which provide the numeric and narrative disclosures from all financial statements and their notes, the Financial Statement Data Sets are more compact.''',
    'version': '2020q4',
    'release_date': '20201231',
    'fields': [
    {
        'adsh':'Accession Number. The 20-character string formed from the 18-digit number assigned by the SEC to each EDGAR submission.',
        'cik':'Central Index Key (CIK). Ten digit number assigned by the SEC to each registrant that submits filings.',
        'name':'Name of registrant. This corresponds to the name of the legal entity as recorded in EDGAR as of the filing date.',
        'sic':'Standard Industrial Classification (SIC). Four digit code assigned by the SEC as of the filing date, indicating the registrant’s type of business.',
        'countryba':'The ISO 3166-1 country of the registrant’s business address.',
        'stprba':'The state or province of the registrant’s business address, if field countryba is US or CA.',
        'cityba':'The city of the registrant’s business address.',
        'zipba':'The zip code of the registrant’s business address.',
        'bas1':'The first line of the street of the registrant’s business address.',
        'bas2':'The second line of the street of the registrant’s business address.',
        'baph':'The phone number of the registrant’s business address.',
        'countryma':'The ISO 3166-1 country of the registrant’s mailing address.',
        'stprma':'The state or province of the registrant’s mailing address, if field countryma is US or CA.',
        'cityma':'The city of the registrant’s mailing address.',
        'zipma':'The zip code of the registrant’s mailing address.',
        'mas1':'The first line of the street of the registrant’s mailing address.',
        'mas2':'The second line of the street of the registrant’s mailing address.',
        'countryinc':'The country of incorporation for the registrant.',
        'stprinc':'The state or province of incorporation for the registrant, if countryinc is US or CA.',
        'ein':'Employee Identification Number, 9 digit identification number assigned by the Internal Revenue Service to business entities operating in the United States.',
        'former':'Most recent former name of the registrant, if any.',
        'changed':'Date of change from the former name, if any.',
        'afs':'Filer status with the SEC at the time of submission:\n\
1-LAF=Large Accelerated,\n\
2-ACC=Accelerated,\n\
3-SRA=Smaller Reporting Accelerated,\n\
4-NON=Non-Accelerated,\n\
5-SML=Smaller Reporting Filer,\n\
NULL=not assigned.',
        'wksi':'Well Known Seasoned Issuer (WKSI). An issuer that meets specific SEC requirements at some point during a 60-day period preceding the date the issuer satisfies its obligation to update its shelf registration statement.',
        'fye':'Fiscal Year End Date, rounded to nearest month-end.',
        'form':'The submission type of the registrant’s filing.',
        'period':'Balance Sheet Date, rounded to nearest month-end.',
        'fy':'Fiscal Year Focus (as defined in EFM Ch. 6).',
        'fp':'Fiscal Period Focus (as defined in EFM Ch. 6) within Fiscal Year. The 10-Q for the 1st, 2nd and 3rd quarters would have a fiscal period focus of Q1, Q2 (or H1), and Q3 (or M9) respectively, and a 10-K would have a fiscal period focus of FY.',
        'filed':'The date of the registrant’s filing with the Commission.',
        'accepted':'The acceptance date and time of the registrant’s filing with the Commission. Filings accepted after 5:30pm EST are considered filed on the following business day.',
        'prevrpt':'Previous Report –TRUE indicates that the submission information was subsequently amended.',
        'detail':'TRUE indicates that the XBRL submission contains quantitative disclosures within the footnotes and schedules at the required detail level (e.g., each amount).',
        'instance':'The name of the submitted XBRL Instance Document (EX-101.INS) type data file. The name often begins with the company ticker symbol.',
        'nciks':'Number of Central Index Keys (CIK) of registrants (i.e., business units) included in the consolidating entity’s submitted filing.',
        'aciks':'Additional CIKs of co-registrants included in  a consolidating entity’s EDGAR submission, separated by spaces. If there are no other co-registrants (i.e., nciks=1), the value of aciks is NULL.  For a very small number of filers, the list of co-registrants is too long to fit in the field.  Where this is the case, PARTIAL will appear at the end of the list indicating that not all co-registrants’ CIKs are included in the field; users should refer to the complete submission file for all CIK information.'
    }]
}
schemaname = 'sec_dera'
tablename = 'sub'

Convert custom metadata content in json format into Pandas DataFrame

In [28]:
df_meta_fields = pd.json_normalize(custom_meta_content, record_path =['fields'], meta=['dataset_key']).convert_dtypes()
df_meta_fields

,adsh,cik,name,sic,countryba,stprba,cityba,zipba,bas1,bas2,...,fy,fp,filed,accepted,prevrpt,detail,instance,nciks,aciks,dataset_key
0,Accession Number. The 20-character string form...,Central Index Key (CIK). Ten digit number assi...,Name of registrant. This corresponds to the na...,Standard Industrial Classification (SIC). Four...,The ISO 3166-1 country of the registrant’s bus...,The state or province of the registrant’s busi...,The city of the registrant’s business address.,The zip code of the registrant’s business addr...,The first line of the street of the registrant...,The second line of the street of the registran...,...,Fiscal Year Focus (as defined in EFM Ch. 6).,Fiscal Period Focus (as defined in EFM Ch. 6) ...,The date of the registrant’s filing with the C...,The acceptance date and time of the registrant...,Previous Report –TRUE indicates that the submi...,TRUE indicates that the XBRL submission contai...,The name of the submitted XBRL Instance Docume...,Number of Central Index Keys (CIK) of registra...,Additional CIKs of co-registrants included in ...,SEC-DERA


In [29]:
df_meta_fields.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 37 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   adsh         1 non-null      string
 1   cik          1 non-null      string
 2   name         1 non-null      string
 3   sic          1 non-null      string
 4   countryba    1 non-null      string
 5   stprba       1 non-null      string
 6   cityba       1 non-null      string
 7   zipba        1 non-null      string
 8   bas1         1 non-null      string
 9   bas2         1 non-null      string
 10  baph         1 non-null      string
 11  countryma    1 non-null      string
 12  stprma       1 non-null      string
 13  cityma       1 non-null      string
 14  zipma        1 non-null      string
 15  mas1         1 non-null      string
 16  mas2         1 non-null      string
 17  countryinc   1 non-null      string
 18  stprinc      1 non-null      string
 19  ein          1 non-null      stri

In [30]:
df_meta_table = pd.json_normalize(custom_meta_content, max_level=0)
df_meta_table.drop('fields', inplace=True, axis=1)
df_meta_table['schema'] = schemaname
df_meta_table = df_meta_table.convert_dtypes()
df_meta_table

,dataset_key,title,description,version,release_date,schema
0,SEC-DERA,SEC DERA Disclosures,The DERA Financial Statement Data Sets provide...,2020q4,20201231,sec_dera


In [31]:
df_meta_table.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   dataset_key   1 non-null      string
 1   title         1 non-null      string
 2   description   1 non-null      string
 3   version       1 non-null      string
 4   release_date  1 non-null      string
 5   schema        1 non-null      string
dtypes: string(6)
memory usage: 176.0 bytes


Open a Trino connection using JWT for authentication

In [32]:
sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ['TRINO_USER'],
    host = os.environ['TRINO_HOST'],
    port = os.environ['TRINO_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ['TRINO_PASSWD']),
    'http_scheme': 'https'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
print("connecting with engine " + str(engine))
connection = engine.connect()

connecting with engine Engine(trino://MichaelTiemannOSC@trino-secure-odh-trino.apps.odh-cl1.apps.os-climate.org:443/)


In [41]:
# Show available schemas to ensure trino connection is set correctly
schema_read = engine.execute('show schemas in osc_datacommons_dev')
for row in schema_read.fetchall():
    print(row)

('aicoe_osc_demo',)
('company_data',)
('default',)
('defaultschema1',)
('demo',)
('eje_test_iceberg',)
('epacems',)
('epacems_y95_al',)
('essd',)
('gleif',)
('information_schema',)
('metastore',)
('metastore_iceberg',)
('osc_corp_data',)
('physical_risk_project',)
('pudl',)
('rmi_utility_transition_hub',)
('sec_dera',)
('team1',)
('team2',)
('testaccessschema1',)
('testdb',)
('urgentem',)
('wri',)
('wri_gppd',)
('wri_gppd_md',)
('wri_test',)


Create ingestion schema based on source data name and remove old tables if necessary

In [42]:
schema_check = engine.execute('create schema if not exists osc_datacommons_dev.' + schemaname)
for row in schema_check.fetchall():
    print(row)

(True,)


In [43]:
table_check = engine.execute('drop table if exists osc_datacommons_dev.' + schemaname + '.' + tablename)
for row in table_check.fetchall():
    print(row)

(True,)


In [46]:
schema = create_table_schema_pairs(df)
tabledef = """
create table if not exists osc_datacommons_dev.{sname}.{tname} (
{schema}
) with (
    format = 'parquet'
    -- partitioning = ARRAY['uuid']
)
""".format(schema=schema, sname=schemaname, tname=tablename)
print(tabledef)


create table if not exists osc_datacommons_dev.sec_dera.sub (
    adsh varchar,
    cik bigint,
    name varchar,
    sic bigint,
    countryba varchar,
    stprba varchar,
    cityba varchar,
    zipba varchar,
    bas1 varchar,
    bas2 varchar,
    baph varchar,
    countryma varchar,
    stprma varchar,
    cityma varchar,
    zipma varchar,
    mas1 varchar,
    mas2 varchar,
    countryinc varchar,
    stprinc varchar,
    ein bigint,
    former varchar,
    changed double,
    afs varchar,
    wksi bigint,
    fye bigint,
    form varchar,
    period bigint,
    fy double,
    fp varchar,
    filed bigint,
    accepted varchar,
    prevrpt bigint,
    detail bigint,
    instance varchar,
    nciks bigint,
    aciks double
) with (
    format = 'parquet'
    -- partitioning = ARRAY['uuid']
)



In [47]:
table_create = engine.execute(tabledef)
for row in table_create.fetchall():
    print(row)

(True,)


In [48]:
# Append data frame to new Trino table 
# this statement should work but receives a TrinoUserError: "This connector does not support creating tables"
# df.to_sql('gppd', con=engine, schema='wri_test', if_exists='append')

In [49]:
list_values = df.values.tolist()
list_values[0]

['0000010254-20-000105',
 10254,
 'EARTHSTONE ENERGY INC',
 1311,
 'US',
 'TX',
 'THE WOODLANDS',
 '77380',
 '1400 WOODLOCH FOREST DRIVE',
 'SUITE 300',
 '281-298-4246',
 'US',
 'TX',
 'THE WOODLANDS',
 '77380',
 '1400 WOODLOCH FOREST DRIVE',
 'SUITE 300',
 'US',
 'DE',
 840592823,
 'BASIC EARTH SCIENCE SYSTEMS INC',
 19920703.0,
 '2-ACC',
 0,
 1231,
 '8-K',
 20200930,
 nan,
 nan,
 20201001,
 '2020-10-01 16:05:00.0',
 0,
 0,
 'form8-kxbbredeterminat_htm.xml',
 1,
 nan]

In [51]:
list_length = len(list_values)
for i in range(list_length):
    joined_values = '(\'' + list_values[i][0] + '\', ' + str(list_values[i][1]) + ', \'' + list_values[i][2] + '\')'
    insert_statement = """INSERT INTO osc_datacommons_dev.{sname}.{tname} 
    VALUES """.format(sname=schemaname, tname=tablename) + joined_values
    print(insert_statement)
    run_statement = engine.execute(insert_statement)
    for row in run_statement.fetchall():
        print(row)

INSERT INTO osc_datacommons_dev.sec_dera.sub 
    VALUES ('0000010254-20-000105', 10254, 'EARTHSTONE ENERGY INC')


TrinoUserError: TrinoUserError(type=USER_ERROR, name=TYPE_MISMATCH, message="Insert query has mismatched column types: Table: [varchar, bigint, varchar, bigint, varchar, varchar, varchar, varchar, varchar, varchar, varchar, varchar, varchar, varchar, varchar, varchar, varchar, varchar, varchar, bigint, varchar, double, varchar, bigint, bigint, varchar, bigint, double, varchar, bigint, varchar, bigint, bigint, varchar, bigint, double], Query: [varchar(20), integer, varchar(21)]", query_id=20211121_181359_00021_adxr6)

In [20]:
dataset_query = ('SELECT * FROM osc_datacommons_dev.{sname}.{tname} limit 10').format(sname=schemaname,tname=tablename)
print(dataset_query)
dataset = engine.execute(dataset_query)
for row in dataset.fetchall():
    print(row)

SELECT * FROM osc_datacommons_iceberg_dev.wri_test.gppd_new limit 10
('Albas', 1.8, '32a98e7e-0f8e-4c76-86bb-7e598928d859')
('Albi', 6.70448, '32a98e7e-0f8e-4c76-86bb-7e598928d859')
('Albias', 2.41, '32a98e7e-0f8e-4c76-86bb-7e598928d859')
('Allaire', 8.2, '32a98e7e-0f8e-4c76-86bb-7e598928d859')


Query Iceberg snapshots for WRI GPPD data set. Snapshots allow having an immutable set of the data at a given time. They are automatically created on every append or removal of data.

In [21]:
snapshot_query = ('SELECT committed_at, snapshot_id, parent_id FROM osc_datacommons_iceberg_dev.{sname}.\"{tname}$snapshots\"').format(sname=schemaname,tname=tablename)
print(snapshot_query)
dataset = engine.execute(snapshot_query)
for row in dataset.fetchall():
    print(row)

SELECT committed_at, snapshot_id, parent_id FROM osc_datacommons_iceberg_dev.wri_test."gppd_new$snapshots"
('2021-11-17 15:09:12.459 UTC', 9025854779685413976, None)
('2021-11-17 15:09:21.143 UTC', 4181945915156604382, 9025854779685413976)
('2021-11-17 15:09:22.030 UTC', 302793385755985542, 4181945915156604382)
('2021-11-17 15:09:22.874 UTC', 4237598940379949314, 302793385755985542)
('2021-11-17 15:09:23.879 UTC', 7161852363068757534, 4237598940379949314)


Create metadata table for schema / dataset level information

In [22]:
# declare variable names for metadata structure in Trino
meta_schema_name = 'metastore_iceberg'
meta_table_name_dataset = 'meta_tables_iceberg'
meta_table_name_fields = 'meta_fields_iceberg'

In [23]:
schema_check = engine.execute('create schema if not exists osc_datacommons_iceberg_dev.' + meta_schema_name)
for row in schema_check.fetchall():
    print(row)

(True,)


In [24]:
table_check = engine.execute('drop table if exists osc_datacommons_iceberg_dev.' + meta_schema_name + '.' + meta_table_name_dataset)
for row in table_check.fetchall():
    print(row)

(True,)


In [25]:
schema_meta_table = create_table_schema_pairs(df_meta_table)
tabledef = """
create table if not exists osc_datacommons_iceberg_dev.{sname}.{tname} (
{schema}
) with (
    format = 'parquet',
    partitioning = ARRAY['dataset_key']
)
""".format(schema=schema_meta_table, sname=meta_schema_name, tname=meta_table_name_dataset)
print(tabledef)


create table if not exists osc_datacommons_iceberg_dev.metastore_iceberg.meta_tables_iceberg (
    dataset_key varchar,
    title varchar,
    description varchar,
    version varchar,
    release_date varchar,
    schema varchar
) with (
    format = 'parquet',
    partitioning = ARRAY['dataset_key']
)



In [26]:
meta_table_create = engine.execute(tabledef)
for row in meta_table_create.fetchall():
    print(row)

(True,)


In [27]:
list_values_meta_table = df_meta_table.values.tolist()
list_values_meta_table[0]

['gppd',
 'Global Power Plant Database',
 'A comprehensive, global, open source database of power plants',
 '1.3.0',
 '20210602',
 'wri_test']

In [28]:
joined_values = '(\'' + list_values_meta_table[0][0] + '\', \'' + list_values_meta_table[0][1] + '\', \'' + list_values_meta_table[0][2] + '\', \'' + list_values_meta_table[0][3] + '\', \'' + list_values_meta_table[0][4] + '\', \'' + list_values_meta_table[0][5] + '\')'
insert_statement = """INSERT INTO osc_datacommons_iceberg_dev.{sname}.{tname} 
VALUES """.format(sname=meta_schema_name,tname=meta_table_name_dataset) + joined_values
print(insert_statement)
run_statement = engine.execute(insert_statement)
for row in run_statement.fetchall():
    print(row)

INSERT INTO osc_datacommons_iceberg_dev.metastore_iceberg.meta_tables_iceberg 
VALUES ('gppd', 'Global Power Plant Database', 'A comprehensive, global, open source database of power plants', '1.3.0', '20210602', 'wri_test')
(1,)


In [29]:
meta_query_table = ('SELECT * FROM osc_datacommons_iceberg_dev.{sname}.{tname} limit 10').format(sname=meta_schema_name,tname=meta_table_name_dataset)
print(meta_query_table)
meta_table_query = engine.execute(meta_query_table)
for row in meta_table_query.fetchall():
    print(row)

SELECT * FROM osc_datacommons_iceberg_dev.metastore_iceberg.meta_tables_iceberg limit 10
('gppd', 'Global Power Plant Database', 'A comprehensive, global, open source database of power plants', '1.3.0', '20210602', 'wri_test')


Create metadata table for fields information

In [30]:
table_check = engine.execute('drop table if exists osc_datacommons_iceberg_dev.' + meta_schema_name + '.' + meta_table_name_fields)
for row in table_check.fetchall():
    print(row)

(True,)


In [31]:
schema_meta_fields = create_table_schema_pairs(df_meta_fields)
tabledef = """
create table if not exists osc_datacommons_iceberg_dev.{sname}.{tname} (
{schema}
) with (
    format = 'parquet',
    partitioning = ARRAY['dataset_key']
)
""".format(schema=schema_meta_fields, sname=meta_schema_name, tname=meta_table_name_fields)
print(tabledef)


create table if not exists osc_datacommons_iceberg_dev.metastore_iceberg.meta_fields_iceberg (
    field_name varchar,
    definition varchar,
    type varchar,
    dataset_key varchar
) with (
    format = 'parquet',
    partitioning = ARRAY['dataset_key']
)



In [32]:
meta_fields_create = engine.execute(tabledef)
for row in meta_fields_create.fetchall():
    print(row)

(True,)


In [33]:
list_values_meta_fields = df_meta_fields.values.tolist()
list_values_meta_fields[0]

['name', 'name of the power plant', 'text', 'gppd']

In [34]:
list_fields_length = len(list_values_meta_fields)
for i in range(list_fields_length):
    joined_values = '(\'' + list_values_meta_fields[i][0] + '\', \'' + list_values_meta_fields[i][1] + '\', \'' + list_values_meta_fields[i][2] + '\', \'' + list_values_meta_fields[i][3] + '\')'
    insert_statement = """INSERT INTO osc_datacommons_iceberg_dev.{sname}.{tname} 
    VALUES """.format(sname=meta_schema_name, tname=meta_table_name_fields) + joined_values
    print(insert_statement)
    run_statement = engine.execute(insert_statement)
    for row in run_statement.fetchall():
        print(row)

INSERT INTO osc_datacommons_iceberg_dev.metastore_iceberg.meta_fields_iceberg 
    VALUES ('name', 'name of the power plant', 'text', 'gppd')
(1,)
INSERT INTO osc_datacommons_iceberg_dev.metastore_iceberg.meta_fields_iceberg 
    VALUES ('capacity_gwh', 'electricity generation in gigawatt-hours', 'number', 'gppd')
(1,)


In [35]:
meta_query_fields = ('SELECT * FROM osc_datacommons_iceberg_dev.{sname}.{tname} limit 10').format(sname=meta_schema_name, tname=meta_table_name_fields)
print(meta_query_fields)
meta_fields_query = engine.execute(meta_query_fields)
for row in meta_fields_query.fetchall():
    print(row)

SELECT * FROM osc_datacommons_iceberg_dev.metastore_iceberg.meta_fields_iceberg limit 10
('name', 'name of the power plant', 'text', 'gppd')
('capacity_gwh', 'electricity generation in gigawatt-hours', 'number', 'gppd')


Update the source data to create a new data set for ingestion

In [36]:
# initialise new data for powerplants and capacity
new_data = {'name':['Albas', 'Albi', 'Albias', 'Allaire'],
        'capacity_gwh':[2.8, 6.90448, 2.20, 7.4]}
# Create DataFrame
new_df = pd.DataFrame(new_data)
# Add a unique identifier to the data set
uid = str(uuid.uuid4())
new_df['uuid'] = uid
# Print the output
new_df = new_df.convert_dtypes()
new_df

,name,capacity_gwh,uuid
0,Albas,2.8,62f65f0d-6dcc-4265-8be1-c181945de9c2
1,Albi,6.90448,62f65f0d-6dcc-4265-8be1-c181945de9c2
2,Albias,2.2,62f65f0d-6dcc-4265-8be1-c181945de9c2
3,Allaire,7.4,62f65f0d-6dcc-4265-8be1-c181945de9c2


In [37]:
new_list_values = new_df.values.tolist()
new_list_values[0]

['Albas', 2.8, '62f65f0d-6dcc-4265-8be1-c181945de9c2']

Ingest new data set

In [38]:
list_length = len(new_list_values)
for i in range(list_length):
    joined_values = '(\'' + new_list_values[i][0] + '\', ' + str(new_list_values[i][1]) + ', \'' + new_list_values[i][2] + '\')'
    insert_statement = """INSERT INTO osc_datacommons_iceberg_dev.{sname}.{tname} 
    VALUES """.format(sname=schemaname, tname=tablename) + joined_values
    print(insert_statement)
    run_statement = engine.execute(insert_statement)
    for row in run_statement.fetchall():
        print(row)

INSERT INTO osc_datacommons_iceberg_dev.wri_test.gppd_new 
    VALUES ('Albas', 2.8, '62f65f0d-6dcc-4265-8be1-c181945de9c2')
(1,)
INSERT INTO osc_datacommons_iceberg_dev.wri_test.gppd_new 
    VALUES ('Albi', 6.90448, '62f65f0d-6dcc-4265-8be1-c181945de9c2')
(1,)
INSERT INTO osc_datacommons_iceberg_dev.wri_test.gppd_new 
    VALUES ('Albias', 2.2, '62f65f0d-6dcc-4265-8be1-c181945de9c2')
(1,)
INSERT INTO osc_datacommons_iceberg_dev.wri_test.gppd_new 
    VALUES ('Allaire', 7.4, '62f65f0d-6dcc-4265-8be1-c181945de9c2')
(1,)


Query data and Iceberg snapshot

In [39]:
print(dataset_query)
dataset = engine.execute(dataset_query)
for row in dataset.fetchall():
    print(row)

SELECT * FROM osc_datacommons_iceberg_dev.wri_test.gppd_new limit 10
('Albi', 6.90448, '62f65f0d-6dcc-4265-8be1-c181945de9c2')
('Allaire', 7.4, '62f65f0d-6dcc-4265-8be1-c181945de9c2')
('Albi', 6.70448, '32a98e7e-0f8e-4c76-86bb-7e598928d859')
('Albias', 2.41, '32a98e7e-0f8e-4c76-86bb-7e598928d859')
('Allaire', 8.2, '32a98e7e-0f8e-4c76-86bb-7e598928d859')
('Albas', 2.8, '62f65f0d-6dcc-4265-8be1-c181945de9c2')
('Albas', 1.8, '32a98e7e-0f8e-4c76-86bb-7e598928d859')
('Albias', 2.2, '62f65f0d-6dcc-4265-8be1-c181945de9c2')


In [40]:
print(snapshot_query)
dataset = engine.execute(snapshot_query)
for row in dataset.fetchall():
    print(row)

SELECT committed_at, snapshot_id, parent_id FROM osc_datacommons_iceberg_dev.wri_test."gppd_new$snapshots"
('2021-11-17 15:09:12.459 UTC', 9025854779685413976, None)
('2021-11-17 15:09:21.143 UTC', 4181945915156604382, 9025854779685413976)
('2021-11-17 15:09:22.030 UTC', 302793385755985542, 4181945915156604382)
('2021-11-17 15:09:22.874 UTC', 4237598940379949314, 302793385755985542)
('2021-11-17 15:09:23.879 UTC', 7161852363068757534, 4237598940379949314)
('2021-11-17 15:10:38.231 UTC', 4462538251865928736, 7161852363068757534)
('2021-11-17 15:10:39.343 UTC', 419656586561256577, 4462538251865928736)
('2021-11-17 15:10:40.250 UTC', 2845024834783547281, 419656586561256577)
('2021-11-17 15:10:41.306 UTC', 3968975231154658376, 2845024834783547281)


Query only the first data set (Iceberg time machine)

In [41]:
past_dataset_query = ('SELECT * FROM osc_datacommons_iceberg_dev.{sname}.\"{tname}@7161852363068757534\"').format(sname=schemaname,tname=tablename)
print(past_dataset_query)
dataset = engine.execute(past_dataset_query)
for row in dataset.fetchall():
    print(row)

SELECT * FROM osc_datacommons_iceberg_dev.wri_test."gppd_new@7161852363068757534"
('Albias', 2.41, '32a98e7e-0f8e-4c76-86bb-7e598928d859')
('Albi', 6.70448, '32a98e7e-0f8e-4c76-86bb-7e598928d859')
('Albas', 1.8, '32a98e7e-0f8e-4c76-86bb-7e598928d859')
('Allaire', 8.2, '32a98e7e-0f8e-4c76-86bb-7e598928d859')


Rollback the second data set 

In [42]:
rollback_request = ('CALL osc_datacommons_iceberg_dev.system.rollback_to_snapshot(\'{sname}\', \'{tname}\', 7161852363068757534)').format(sname=schemaname,tname=tablename)
print(rollback_request)
dataset = engine.execute(rollback_request)
for row in dataset.fetchall():
    print(row)

CALL osc_datacommons_iceberg_dev.system.rollback_to_snapshot('wri_test', 'gppd_new', 7161852363068757534)
(True,)


Query the full table again, the second data set has been rolled back

In [43]:
print(dataset_query)
dataset = engine.execute(dataset_query)
for row in dataset.fetchall():
    print(row)

SELECT * FROM osc_datacommons_iceberg_dev.wri_test.gppd_new limit 10
('Albi', 6.70448, '32a98e7e-0f8e-4c76-86bb-7e598928d859')
('Albias', 2.41, '32a98e7e-0f8e-4c76-86bb-7e598928d859')
('Allaire', 8.2, '32a98e7e-0f8e-4c76-86bb-7e598928d859')
('Albas', 1.8, '32a98e7e-0f8e-4c76-86bb-7e598928d859')
